In [1]:
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
# from google.colab import files
import pyarrow.parquet as pq
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_full = pd.read_parquet('data/df_full.parquet')
#Drop some column
tns=df_full.copy()
tns=tns.drop(columns=['center','product_json','product','qty'])
tns['payment_date']=pd.to_datetime(tns['payment_date'])


In [3]:
tns[tns['paid_amount']<0]

,payment_date,ent,total_amount,discount,paid_amount,trans_origin_type,0C1CC4,0C4C43,0C4C4C,0C4C4Q,...,XC4CCE,XC4CCQ,XC4CCW,XCECC4,XCECCJ,XCECCQ,XCJCC3,XCJCCZ,XCQCC4,XP9LI
97312,2022-02-03,TCC43RWW33Y,54000.0,500000.0,-446000.0,online,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
560160,2022-07-09,TCCE4ZRW1EK,189000.0,191500.0,-2500.0,online,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
717896,2022-09-19,TCC4ECEZJWY,2442000.0,2502500.0,-60500.0,online,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
718616,2022-09-19,TCC4QCEJCZB,329000.0,331500.0,-2500.0,online,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
874830,2022-11-14,TCC4ECEZJWY,2472000.0,2502500.0,-30500.0,online,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# drop df_full[df_full['paid_amount']<0
tns=tns[tns['paid_amount']>0]
tns=tns.reset_index(drop=True)
tns.head()

,payment_date,ent,total_amount,discount,paid_amount,trans_origin_type,0C1CC4,0C4C43,0C4C4C,0C4C4Q,...,XC4CCE,XC4CCQ,XC4CCW,XCECC4,XCECCJ,XCECCQ,XCJCC3,XCJCCZ,XCQCC4,XP9LI
0,2021-01-01,TCC43RQQZ3B,1627000.0,0.0,1627000.0,online,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-01-01,TCC4Q4JZ4Q0,594000.0,0.0,594000.0,online,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-01-01,TCC4RJ11QE7,468000.0,0.0,468000.0,online,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021-01-01,TCC434QRCJY,341000.0,0.0,341000.0,online,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-01-01,TCC43RRCE10,189000.0,0.0,189000.0,online,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create Features

In [5]:
def split_dataframe_by_quarter_and_year(dataframe, date_column):
    # Ensure the date_column is in datetime format
    dataframe[date_column] = pd.to_datetime(dataframe[date_column])

    # Group by year and quarter
    dataframe['quarter'] = dataframe[date_column].dt.to_period('Q')
    grouped = dataframe.groupby(['quarter', dataframe[date_column].dt.year])

    # Initialize the result dictionary
    result_dict = {}

    # Iterate over the groups and store DataFrames in the result dictionary
    for (quarter, year), group in grouped:
        key = f"{year}-{quarter.quarter}"
        result_dict[key] = group.copy()

    return result_dict

In [6]:
# Split the DataFrame
tns_split = split_dataframe_by_quarter_and_year(tns, 'payment_date')
tns_split

{'2021-1':               payment_date          ent  total_amount  discount  paid_amount  \
 0      2021-01-01 00:00:00  TCC43RQQZ3B     1627000.0       0.0    1627000.0   
 1      2021-01-01 00:00:00  TCC4Q4JZ4Q0      594000.0       0.0     594000.0   
 2      2021-01-01 00:00:00  TCC4RJ11QE7      468000.0       0.0     468000.0   
 3      2021-01-01 00:00:00  TCC434QRCJY      341000.0       0.0     341000.0   
 4      2021-01-01 00:00:00  TCC43RRCE10      189000.0       0.0     189000.0   
 ...                    ...          ...           ...       ...          ...   
 603644 2021-03-31 22:28:00  TCC4RCC1JZP       50000.0       0.0      50000.0   
 603645 2021-03-31 22:29:00  TCC4R4C3CCU       50000.0       0.0      50000.0   
 603646 2021-03-31 22:41:00  TCC4R4C3J45       50000.0       0.0      50000.0   
 603647 2021-03-31 23:12:00  TCC41Q1QZQP       50000.0       0.0      50000.0   
 603648 2021-03-31 23:40:00  TCC41Q1W3WB       50000.0       0.0      50000.0   
 
        trans_or

In [7]:
df_1Q22 = tns_split['2022-1']
df_2Q22 = tns_split['2022-2']
df_3Q22 = tns_split['2022-3']
df_4Q22 = tns_split['2022-4']
df_1Q23 = tns_split['2023-1']
df_2Q23 = tns_split['2023-2']

In [8]:
print(f'1Q22: {df_1Q22.payment_date.min(), df_1Q22.payment_date.max()}')
print(f'2Q22: {df_2Q22.payment_date.min(), df_2Q22.payment_date.max()}')
print(f'3Q22: {df_3Q22.payment_date.min(), df_3Q22.payment_date.max()}')
print(f'4Q22: {df_4Q22.payment_date.min(), df_4Q22.payment_date.max()}')
print(f'1Q23: {df_1Q23.payment_date.min(), df_1Q23.payment_date.max()}')
print(f'2Q23: {df_2Q23.payment_date.min(), df_2Q23.payment_date.max()}')

1Q22: (Timestamp('2022-01-01 00:00:00'), Timestamp('2022-03-31 00:00:00'))
2Q22: (Timestamp('2022-04-01 00:00:00'), Timestamp('2022-06-30 00:00:00'))
3Q22: (Timestamp('2022-07-01 00:00:00'), Timestamp('2022-09-30 00:00:00'))
4Q22: (Timestamp('2022-10-01 00:00:00'), Timestamp('2022-12-31 00:00:00'))
1Q23: (Timestamp('2023-01-01 00:00:00'), Timestamp('2023-03-31 00:00:00'))
2Q23: (Timestamp('2023-04-01 00:00:00'), Timestamp('2023-06-30 00:00:00'))


In [9]:
def create_features(df_rolling):
    df_rolling = df_rolling.sort_values(by='payment_date')
    df_rolling = df_rolling.reset_index(drop=True)

    features_rolling = pd.DataFrame(df_rolling.groupby('ent')['ent'].count())
    features_rolling.rename(columns={'ent': 'frequency'}, inplace=True)

    features_rolling['startdate'] = list(df_rolling.groupby('ent')['payment_date'].min())
    features_rolling['lastdate'] = list(df_rolling.groupby('ent')['payment_date'].max())
    features_rolling['lastdate'] = pd.to_datetime(features_rolling['lastdate'])

    features_rolling['total_time'] = ((features_rolling['lastdate'] - features_rolling['startdate']).dt.days) + 1
    features_rolling['mtbp'] = features_rolling['total_time'] / features_rolling['frequency']

    #####
    df = df_rolling.copy()
    one_hot_encoded = pd.get_dummies(df['trans_origin_type'], prefix='trans_type')
    df = pd.concat([df.iloc[:, :6], one_hot_encoded], axis=1)
    df = df.drop(columns=['trans_origin_type'])

    df['Avg_Discount'] = (1 - (df['paid_amount'] / df['total_amount'])) * 100

    features = df.groupby('ent')['paid_amount'].sum().reset_index()

    features['frequency'] = list(df.groupby('ent')['ent'].count())
    features['Avg_spend'] = features['paid_amount'] / features['frequency']
    features = features.merge(df.groupby('ent')['Avg_Discount'].mean().reset_index(), on='ent', how='left')

    features['startdate'] = list(df.groupby('ent')['payment_date'].min())
    features['lastdate'] = list(df.groupby('ent')['payment_date'].max())
    features['lastdate'] = pd.to_datetime(features['lastdate'])

    features['total_time'] = ((features['lastdate'] - features['startdate']).dt.days) + 1

    # Merge the 'mtbp' from previous rolling features
    features = features.merge(features_rolling['mtbp'], on='ent', how='left')

    features = features.merge(df.groupby('ent')['trans_type_offline'].sum().reset_index(), on='ent', how='left')
    features = features.merge(df.groupby('ent')['trans_type_online'].sum().reset_index(), on='ent', how='left')
    # features['offline_ratio'] = features['trans_type_offline'] / (features['trans_type_offline'] + features['trans_type_online'])
    features['online_ratio'] = features['trans_type_online'] / (features['trans_type_offline'] + features['trans_type_online'])

    features.drop(columns=['paid_amount', 'startdate', 'lastdate', 'total_time', 'trans_type_offline', 'trans_type_online'], inplace=True)
    features['Avg_Discount'] = features['Avg_Discount'].fillna(0)
    
    return features


In [10]:
df_full_1Q22 = create_features(df_1Q22)
df_full_2Q22 = create_features(df_2Q22)
df_full_3Q22 = create_features(df_3Q22)
df_full_4Q22 = create_features(df_4Q22)
df_full_1Q23 = create_features(df_1Q23)
df_full_2Q23 = create_features(df_2Q23)

In [11]:
df_full_2Q23.head()

,ent,frequency,Avg_spend,Avg_Discount,mtbp,online_ratio
0,8CCCCCC4R1F,1,367500.0,0.675676,1.0,1.00
1,BC14413K,1,1118000.0,8.210181,1.0,0.00
2,BCCCCRE3C4F,1,335000.0,0.000000,1.0,1.00
3,BCCCCREW3WY,4,469750.0,23.495370,20.0,0.75
4,BCE4JERY,4,1687250.0,0.000000,15.0,0.00


In [12]:
df_full_1Q22.to_parquet('data/df_1Q22.parquet')
df_full_2Q22.to_parquet('data/df_2Q22.parquet')
df_full_3Q22.to_parquet('data/df_3Q22.parquet')
df_full_4Q22.to_parquet('data/df_4Q22.parquet')
df_full_1Q23.to_parquet('data/df_1Q23.parquet')
df_full_2Q23.to_parquet('data/df_2Q23.parquet')